# Tutorial 5 - Advanced quantum programming

Note: we depend on the `pulser` library for performing quantum programming and you should have knowledge of the library and neutral-atom quantum programming. 

By design, a quantum solver is composed of two main quantum components for solving a problem:

- an embedder,  i.e. a mechanism used to customize the layout of neutral atoms on the quantum device.
- a pulse shaper, i.e. a mechanism used to customize the laser pulse to which the neutral atoms are subjected during the execution of the quantum algorithm.



These can be specified in the `SolverConfig`. Several embedders and pulse shapers are already implemented, but for research purposes, we allow the possibility to design our own custom embedder and pulse shapers. This tutorial dives into currently available components, as well as designing custom ones.

## Embedder

### Default

When instantiating a `SolverConfig`, a default embedder is already made available. To access the resulting embedding from a solver, simply call the `embedding` method as follows:

In [1]:
from mis import MISSolver, MISInstance, SolverConfig, BackendConfig, BackendType


from networkx import erdos_renyi_graph

# User can fix the seed for reproducibility 
seed = 0
graph = erdos_renyi_graph(n=25, p=0.4, seed=seed)
instance = MISInstance(graph)

qutip_config = BackendConfig(
    backend = BackendType.QUTIP
)

config = SolverConfig(backend = qutip_config)
solver = MISSolver(instance, config)


geometry = solver.embedding()

# draw the register
# geometry.draw()

### Custom

To design our own embedding method, we need to define a class inhereting from `mis.pipeline.embedder.BaseEmbedder` and implement an `embed` method as follows: 

In [2]:
from mis.pipeline.embedder import BaseEmbedder
from pulser import Register
from qoolqit._solvers.backends import BaseBackend
from mis.pipeline.layout import Layout

class DefaultEmbedder(BaseEmbedder):
    """ The DefaultEmbedder class available in mis.
    """

    def embed(self, instance: MISInstance, config: SolverConfig, backend: BaseBackend) -> Register:
        device = backend.device()

        # Use Layout helper to get rescaled coordinates and interaction graph
        layout = Layout.from_device(data=instance, device=device)

        # Finally, prepare register.
        return Register(
            qubits={f"q{node}": pos for (node, pos) in layout.coords.items()}
        )

custom_config = SolverConfig(backend = qutip_config, embedder=DefaultEmbedder())

In [3]:
custom_solver = MISSolver(instance, custom_config)
default_geometry = custom_solver.embedding()

# draw the register
# default_geometry.draw()

## Pulse shaper

### Default

When instantiating a `SolverConfig`, a default pulse shaper is already made available. To access the resulting embedding from a solver, simply call the `pulse` method as follows:

In [4]:
default_pulse = solver.pulse(solver.embedding())
default_pulse

Pulse(amp=InterpolatedWaveform(Points: (0, 1e-09), (3000, 15.71), (5999, 1e-09), Interpolator=PchipInterpolator) rad/µs, detuning=InterpolatedWaveform(Points: (0, -125.7), (3000, 0), (5999, 125.7), Interpolator=PchipInterpolator) rad/µs, phase=0, post_phase_shift=0)

### Custom

To design our own pulse shaping method, we need to define a class inhereting from `mis.pipeline.pulse.BasePulseShaper` and implement an `generate` method as follows: 

In [5]:
from mis.pipeline.pulse import BasePulseShaper, DefaultPulseShaper
from pulser import Pulse

class ConstantPulseShaper(BasePulseShaper):
    """
    We simply return a constant pulse.
    """

    def generate(
        self, config: SolverConfig, register: Register, backend: BaseBackend, instance: MISInstance
    ) -> Pulse:
        import numpy as np
        return Pulse.ConstantPulse(1000, np.pi, 0, 0)

In [6]:
constant_pulse_config = SolverConfig(backend = qutip_config, pulse_shaper=ConstantPulseShaper())
constant_pulse_solver = MISSolver(instance, constant_pulse_config)
constant_pulse = constant_pulse_solver.pulse(constant_pulse_solver.embedding())
constant_pulse

Pulse(amp=ConstantWaveform(1000 ns, 3.14) rad/µs, detuning=ConstantWaveform(1000 ns, 0) rad/µs, phase=0, post_phase_shift=0)